In [1]:
import copy
import glob
import importlib
import itertools
import math
import operator
import os
import random
import time
from os.path import *
import psutil
import shutil

import keras.layers as layers
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import cnn_builder as cbuild
import cnn_runner as crun
import config
import ddpg.env as denv
import ddpg.main as qmain
import ddpg.learning_nets as ln
import ddpg.task_nets as tn
import dr_methods as drm
import seg_methods as sm
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import niftiutils.visualization as vis
import voi_methods as vm

%matplotlib inline
np.set_printoptions(3, suppress=True)
#np.random.seed(0)

C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
def memory():
    pid = os.getpid()
    py = psutil.Process(pid)
    gigs = py.memory_info()[0]/2.**30
    print('Memory use:', gigs)
    return gigs

In [3]:
importlib.reload(cbuild)
C = config.Config()
dqn_generator = cbuild._train_gen_ddpg()

In [ ]:
importlib.reload(qmain)
Q = qmain.CRSNet(1.)
Q.train(dqn_generator, verbose=True)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Episode 0 Replay self.q_buffer 367
D:\Etiology\imgs\full_imgs\4267520.npy
	Action [0.151 0.075 0.554 0.16  0.524 0.63  0.703 0.412 0.093 0.074] Reward -0.0
	Action [0.316 0.67  0.533 0.851 0.114 0.245 0.491 0.494 0.561 0.855] Reward -0.3
	Action [0.591 0.548 0.291 0.474 0.453 0.516 0.662 0.019 0.775 0.933] Reward -0.9
	Action [0.045 0.937 0.85  0.372 0.419 0.686 0.225 0.257 0.577 0.396] Reward -0.1
	Action [0.216 0.019 0.577 0.767 0.467 0.145 0.835 0.697 0.518 0.918] Reward -0.2
	Action [0.433 0.95  0.225 0.808 0.552 0.751 0.262 0.866 0.281 0.48 ] Reward -1.2
	Action [0.796 0.661 0.825 0.391 0.405 0.422 0.327 0.894 0.198 0.153] Reward -0.5
	Action [0.405 0.876 0.788 0.635 0.402 0.824 0.165 0.596 0.68  0.13 ] Reward -1.0
	Action [0.899 0.652 0.057 0.546 0.959 0.963 0.953 0.508 0.48  0.404] Reward -

D:\Etiology\imgs\full_imgs\4651151.npy
D:\Etiology\imgs\full_imgs\5353048.npy
D:\Etiology\imgs\full_imgs\4782059.npy
D:\Etiology\imgs\full_imgs\3613398.npy
D:\Etiology\imgs\full_imgs\4231650.npy
D:\Etiology\imgs\full_imgs\4308573.npy
D:\Etiology\imgs\full_imgs\4194658.npy
D:\Etiology\imgs\full_imgs\4854319.npy
D:\Etiology\imgs\full_imgs\5160536.npy
D:\Etiology\imgs\full_imgs\4122429.npy
D:\Etiology\imgs\full_imgs\4854319.npy
D:\Etiology\imgs\full_imgs\4800749.npy
D:\Etiology\imgs\full_imgs\5226445.npy
D:\Etiology\imgs\full_imgs\3691024.npy
D:\Etiology\imgs\full_imgs\4149378.npy
	Action [0.829 0.336 0.099 0.95  0.136 0.395 0.08  0.185 0.115 0.207] Reward -0.2
	Action [0.804 0.405 0.447 0.987 0.948 0.906 0.053 0.173 0.02  0.859] Reward 0.1
	Action [0.953 0.024 0.469 0.589 0.908 0.894 0.127 0.013 0.162 0.124] Reward 0.6
	Action [0.56  0.304 0.203 0.965 0.926 0.439 0.021 0.023 0.086 0.896] Reward 0.5
	Action [0.74  0.042 0.015 0.805 0.223 0.363 0.024 0.037 0.024 0.956] Reward -1.0
Memory u

D:\Etiology\imgs\full_imgs\4090578.npy
	Action [0.189 0.142 0.292 0.981 0.919 0.221 0.12  0.923 0.013 0.996] Reward -1.0
Memory use: 4.503162384033203
TOTAL REWARD: -1.0 (1 steps)

Episode 17 Replay self.q_buffer 510
D:\Etiology\imgs\full_imgs\4365804.npy
D:\Etiology\imgs\full_imgs\5532336.npy
D:\Etiology\imgs\full_imgs\4366235.npy
D:\Etiology\imgs\full_imgs\4072335.npy
D:\Etiology\imgs\full_imgs\4067703.npy
D:\Etiology\imgs\full_imgs\4072335.npy
D:\Etiology\imgs\full_imgs\4250455.npy
	Action [0.854 0.51  0.135 0.601 0.649 0.164 0.384 0.897 0.07  0.994] Reward -1.0
Memory use: 4.502368927001953
TOTAL REWARD: -1.0 (1 steps)

Episode 18 Replay self.q_buffer 511
D:\Etiology\imgs\full_imgs\4947882.npy
D:\Etiology\imgs\full_imgs\4940582.npy
D:\Etiology\imgs\full_imgs\651494.npy
D:\Etiology\imgs\full_imgs\4072335.npy
D:\Etiology\imgs\full_imgs\4782059.npy
D:\Etiology\imgs\full_imgs\4165057.npy
D:\Etiology\imgs\full_imgs\4875130.npy
D:\Etiology\imgs\full_imgs\4973789.npy
	Action [0.844 0.028 

	Action [0.522 0.094 0.09  0.872 0.772 0.943 0.104 0.461 0.01  0.939] Reward -2.4
	Action [0.18  0.41  0.355 0.621 0.906 0.746 0.03  0.877 0.037 0.938] Reward 0.6
	Action [0.18  0.057 0.018 0.869 0.609 0.784 0.035 0.986 0.045 0.939] Reward -0.7
	Action [0.844 0.217 0.096 0.901 0.862 0.767 0.05  0.956 0.217 0.778] Reward -1.8
	Action [0.857 0.139 0.256 0.876 0.554 0.713 0.019 0.499 0.068 0.936] Reward -0.8
	Action [0.58  0.045 0.071 0.924 0.537 0.625 0.323 0.982 0.02  0.803] Reward -0.6
	Action [0.35  0.025 0.132 0.825 0.768 0.93  0.018 0.927 0.095 0.66 ] Reward -0.4
	Action [0.854 0.068 0.351 0.295 0.972 0.561 0.047 0.545 0.099 0.947] Reward -0.5
	Action [0.947 0.202 0.31  0.826 0.67  0.47  0.087 0.981 0.275 0.823] Reward -0.3
	Action [0.969 0.864 0.204 0.127 0.529 0.609 0.08  0.865 0.15  0.852] Reward -0.4
	Action [0.743 0.173 0.606 0.681 0.797 0.565 0.275 0.941 0.056 0.284] Reward 1.7
	Action [0.988 0.397 0.025 0.458 0.39  0.497 0.016 0.938 0.092 0.963] Reward -1.0
Memory use: 2.6534

D:\Etiology\imgs\full_imgs\4162152.npy
D:\Etiology\imgs\full_imgs\651494.npy
D:\Etiology\imgs\full_imgs\4267520.npy
D:\Etiology\imgs\full_imgs\4487803.npy
	Action [0.659 0.047 0.233 0.813 0.979 0.933 0.173 0.662 0.009 0.731] Reward -1.4
	Action [0.983 0.313 0.33  0.961 0.953 0.831 0.013 0.773 0.089 0.977] Reward -1.0
Memory use: 2.4864540100097656
TOTAL REWARD: -2.4 (2 steps)

Episode 41 Replay self.q_buffer 653
	Action [0.298 0.058 0.075 0.956 0.864 0.803 0.178 0.335 0.012 0.887] Reward -1.3
	Action [0.97  0.278 0.643 0.898 0.249 0.92  0.086 0.951 0.033 0.656] Reward 0.4
	Action [0.387 0.279 0.371 0.876 0.893 0.982 0.128 0.939 0.025 0.75 ] Reward 0.7
	Action [0.986 0.178 0.146 0.853 0.468 0.868 0.101 0.372 0.12  0.27 ] Reward -0.9
	Action [0.831 0.81  0.053 0.608 0.933 0.85  0.014 0.942 0.15  0.991] Reward -1.0
Memory use: 2.508838653564453
TOTAL REWARD: -2.1 (5 steps)

Episode 42 Replay self.q_buffer 658
D:\Etiology\imgs\full_imgs\4162152.npy
D:\Etiology\imgs\full_imgs\4547885.npy
D:

In [ ]:
Q.save_models()

In [6]:
shutil.rmtree(C.model_dir)

In [7]:
os.makedirs(C.model_dir)

In [24]:
cropI = np.random.normal(size=(1,*C.dims,3))

crop_true_seg = np.random.normal(size=(1,*C.dims,3))
true_cls = np.random.uniform(size=(1,3))

y_true = np.random.normal(size=(1,*C.dims,3))
y_pred = np.random.normal(size=(1,*C.dims,4))

loss_layer = train_model.layers[-1]

In [25]:
for _ in range(10):
    print(loss_layer.get_loss(0, y_true, y_pred, C.loss_weights[1:]))
    memory();

-0.05015051
Memory use: 2.0087203979492188
-0.050445613
Memory use: 2.0087203979492188
-0.050510425
Memory use: 2.008716583251953
-0.049581293
Memory use: 2.008716583251953
-0.049884304
Memory use: 2.008716583251953
-0.048125546
Memory use: 2.008716583251953
-0.049017023
Memory use: 2.008716583251953
-0.04843785
Memory use: 2.0087203979492188
-0.050015423
Memory use: 2.0087203979492188
-0.049520925
Memory use: 2.0087203979492188


In [ ]:
importlib.reload(tn)
for _ in range(10):
    log_vars = [K.get_value(x)[0] for x in train_model.layers[-1].log_vars]
    loss = np.sum(np.exp(-log_vars[0]) * K.get_value(tn.hetero_cls_loss(y_true, y_pred)) + log_vars[0], -1)
    print(np.mean(loss))
    memory();

In [30]:
A = prediction_model.predict(cropI)

In [ ]:
for _ in range(10):
    train_model.train_on_batch([cropI, crop_true_seg, true_cls], None)
    A=prediction_model.predict(cropI);
    print(A[1])
    memory();

In [ ]:
importlib.reload(qmain)
Q = qmain.CRSNet(1.)

replay_conf = {'size': 10000,
        'learn_start': 100,
        'partition_num': 100,
        'total_step': 10000,
        'batch_size': 4}
BATCH_SIZE = replay_conf["batch_size"]

Q.load_models(replay_conf)

In [ ]:
Q.critic.model.summary()

In [ ]:
Q.actor.model.summary()

In [ ]:
Q.env.pred_model.summary()

In [ ]:
importlib.reload(qmain)
Q = qmain.CRSNet(.5)
img, true_seg, true_cls = next(dqn_generator)
seg, seg_var, cls = Q.run(img)

In [ ]:
vis.draw_slices(img)

In [ ]:
vis.draw_slices(true_seg)

In [ ]:
vis.draw_slices(seg, normalize=[0,.05])

In [ ]:
hist = M.fit_generator(gen, steps_per_epoch=2, epochs=2)

prediction_model = get_prediction_model()
trainable_model = get_trainable_model(prediction_model)
trainable_model.compile(optimizer='adam', loss=None)
assert len(trainable_model.layers[-1].trainable_weights) == 2  # two log_vars, one for each output
assert len(trainable_model.losses) == 1
hist = trainable_model.fit([X, Y1, Y2], nb_epoch=nb_epoch, batch_size=batch_size, verbose=0)

[np.exp(K.get_value(log_var[0]))**0.5 for log_var in trainable_model.layers[-1].log_vars]

In [ ]:
importlib.reload(config)
importlib.reload(hf)
importlib.reload(cbuild)
importlib.reload(crun)
C = config.Config('etiology')
T = config.Hyperparams()
T.get_best_hyperparams()
T.epochs = 30
T.steps_per_epoch = 300

In [ ]:
importlib.reload(cbuild)
importlib.reload(dqna)
#dqn_generator = cbuild._train_gen_dqn([])
agent = dqna.train_dqn(dqn_generator, agent)

In [ ]:
agent.cls_model.fit(cls_generator)

In [ ]:
T.padding = ['same', 'same']
T.pool_sizes = [(2,2,2),(2,2,2)]
T.f = [64,64,64,64,64,64,64]
T.skip_con = True
T.epochs = 20

In [ ]:
drm.dcm2npy_batch(acc_nums=["E100113043"])
#vm.reset_accnum('E105464882')

In [ ]:
vm.plot_check(2, "E102088195");

In [ ]:
vm.xref_dirs_with_excel()

In [ ]:
crun.run_fixed_hyperparams([C], hyperparams=T)#C_list)

In [ ]:
importlib.reload(cbuild)
model = cbuild.build_cnn_hyperparams(T)
model.summary(line_length=120)

In [ ]:
importlib.reload(cbuild)
#Z_reader = ['E103312835_1','12823036_0','12569915_0','E102093118_0','E102782525_0','12799652_0','E100894274_0','12874178_3','E100314676_0','12842070_0','13092836_2','12239783_0','12783467_0','13092966_0','E100962970_0','E100183257_1','E102634440_0','E106182827_0','12582632_0','E100121654_0','E100407633_0','E105310461_0','12788616_0','E101225606_0','12678910_1','E101083458_1','12324408_0','13031955_0','E101415263_0','E103192914_0','12888679_2','E106096969_0','E100192709_1','13112385_1','E100718398_0','12207268_0','E105244287_0','E102095465_0','E102613189_0','12961059_0','11907521_0','E105311123_0','12552705_0','E100610622_0','12975280_0','E105918926_0','E103020139_1','E101069048_1','E105427046_0','13028374_0','E100262351_0','12302576_0','12451831_0','E102929168_0','E100383453_0','E105344747_0','12569826_0','E100168661_0','12530153_0','E104697262_0']
X_test, Y_test, train_generator, num_samples, train_orig, Z = cbuild.get_cnn_data(n=4)#, Z_test_fixed=Z_reader)
Z_test, Z_train_orig = Z
X_train_orig, Y_train_orig = train_orig
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [ ]:
model.save(join(C.model_dir, "model_.hdf5"))

In [ ]:
model.predict(X_train_orig[20:40])

In [ ]:
Y_train_orig[:20]

In [ ]:
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [ ]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#save_output(Z_test, y_pred, y_true)

print(accuracy_score(y_true, y_pred))
#y_true_simp, y_pred_simp, _ = cnna.merge_classes(y_true, y_pred)
#print(accuracy_score(y_true_simp, y_pred_simp))